In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse
%matplotlib inline
seed = 42
import os
os.environ['OMP_NUM_THREADS'] = '4'
import json


In [18]:
dd = 'D:/Users/sebastian.castillo/PycharmProjects/research_tools/data/'
os.listdir(dd)

['2D_2_clases_simple.csv',
 '2D_6_clases_hard.csv',
 '2D_circulos.csv',
 '2D_moons.csv',
 '2D_not_linear_clases.csv',
 '2D_unbalanced.csv',
 'boston.csv',
 'breast_cancer.csv',
 'breast_cancer_radius.csv',
 'consumo_nafta.csv',
 'corpus_fallosmetdat.json',
 'corpus_textosfallos.json',
 'diabetes.csv',
 'iris.csv',
 'MNIST.npy',
 'MNIST_labels.npy',
 'study_regression_small.csv',
 'titanic.csv',
 'weatherAUS.csv',
 'who_life_expectancy.csv',
 'winequality-red.csv',
 'winequality-white.csv']

In [24]:
#open
fpath = dd + 'corpus_textosfallos.json'
dpath = dd + 'corpus_fallosmetdat.json'
f = open(fpath , encoding="utf8")
d = open(dpath , encoding="utf8")
fallos = json.load(f)
fdatos = json.load(d)
#explorar fdatos
df = pd.DataFrame(fdatos)

In [25]:
# Filtrando bd para trabajar con categorías con observaciones
df['materia'].value_counts()
df['tipo_fallo'].value_counts()
materias_elegidas = ['CIVIL Y COMERCIAL','PENAL','LABORAL']
tipo_felegidos = ["Sentencia Interlocutoria", 'Sentencia Definitiva']
dfxmat = df[df['materia'].notna()]
dfxmat = df[df['materia'].isin(materias_elegidas)]
dfxmat['materia'].value_counts()

CIVIL Y COMERCIAL    1645
PENAL                1507
LABORAL               738
Name: materia, dtype: int64

In [26]:
dfxmat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3890 entries, 0 to 4298
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   indice                3890 non-null   int64 
 1   textos_raw_len        3890 non-null   int64 
 2   materia               3890 non-null   object
 3   fecha                 3890 non-null   object
 4   tipo_fallo            3890 non-null   object
 5   tribunal_emisor       3890 non-null   object
 6   id_causa              3890 non-null   object
 7   fecha_res             3890 non-null   object
 8   nro_reg_int           3890 non-null   object
 9   caratula              3890 non-null   object
 10  caratula_pub          3890 non-null   object
 11  magistrados_votantes  3890 non-null   object
 12  tribunal_origen       3890 non-null   object
 13  nnf                   3890 non-null   object
 14  observ                3890 non-null   object
 15  sent_anulada          3890 non-null   

In [31]:
selec_fallos = dfxmat[['indice']]

## Filtrando bd fallos según las materias elegidas

In [43]:
bdfallos = pd.DataFrame(fallos)

In [47]:
bdfallos = bdfallos[bdfallos.index.isin(dfxmat.index), :]

TypeError: '(array([ True,  True,  True, ...,  True,  True,  True]), slice(None, None, None))' is an invalid key

In [45]:
bdfallos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3890 entries, 0 to 4298
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   indice        3890 non-null   int64 
 1   textos_fallo  3890 non-null   object
dtypes: int64(1), object(1)
memory usage: 91.2+ KB


In [42]:
bdfallos['textos_fallo'][1]

KeyError: 1

## Missing value

In [ ]:
k = pd.DataFrame()
k['train'] = train.isnull().sum()
k['test'] = test.isnull().sum()
k

In [ ]:
test[test['comment_text'].isnull()]

In [ ]:
test.fillna(' ',inplace=True)
gc.collect()

## Text preprosesing

[source: ](https://www.kaggle.com/him4318/easy-and-fast-lb-044) 

Term Frequency Inverse Document  Frequency Vectorizer 

In [ ]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
vect_char = TfidfVectorizer(max_features=40000, lowercase=True, analyzer='char',
                        stop_words= 'english',ngram_range=(3,6),dtype=np.float32)

In [ ]:
# Word ngram vector
tr_vect = vect_word.fit_transform(train['comment_text'])
ts_vect = vect_word.transform(test['comment_text'])

# Character n gram vector
tr_vect_char = vect_char.fit_transform(train['comment_text'])
ts_vect_char = vect_char.transform(test['comment_text'])
gc.collect()

In [ ]:
X = sparse.hstack([tr_vect, tr_vect_char])
x_test = sparse.hstack([ts_vect, ts_vect_char])

In [ ]:
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = train[target_col]
del tr_vect, ts_vect, tr_vect_char, ts_vect_char
gc.collect()

## Model

In [ ]:
prd = np.zeros((x_test.shape[0],y.shape[1]))
cv_score =[]
for i,col in enumerate(target_col):
    lr = LogisticRegression(C=2,random_state = i,class_weight = 'balanced')
    print('Building {} model for column:{''}'.format(i,col)) 
    lr.fit(X,y[col])
    #cv_score.append(lr.score)
    prd[:,i] = lr.predict_proba(x_test)[:,1]

## Model Validation on train data set

In [ ]:
col = 'identity_hate'
print("Column:",col)
pred =  lr.predict(X)
print('\nConfusion matrix\n',confusion_matrix(y[col],pred))
print(classification_report(y[col],pred))

## Roc AUC curve

In [ ]:
col = 'identity_hate'
print("Column:",col)
pred_pro = lr.predict_proba(X)[:,1]
frp,trp,thres = roc_curve(y[col],pred_pro)
auc_val =auc(frp,trp)
plt.figure(figsize=(14,10))
plt.plot([0,1],[0,1],color='b')
plt.plot(frp,trp,color='r',label= 'AUC = %.2f'%auc_val)
plt.legend(loc='lower right')
plt.xlabel('True positive rate')
plt.ylabel('False positive rate')
plt.title('Reciever Operating Characteristic')

## submission

In [ ]:
prd_1 = pd.DataFrame(prd,columns=y.columns)
submit = pd.concat([test['id'],prd_1],axis=1)
#submit.to_csv('toxic_lr.csv.gz',compression='gzip',index=False)
submit.to_csv('toxic_lr.csv',index=False)
submit.head()